In [ ]:
from pathlib import Path
import pickle
import warnings

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import rasterio as rio
from rasterio.features import rasterize
from rasterio.mask import mask
from rasterio.windows import from_bounds
from rasterio.warp import Resampling, reproject

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
import sys
from pathlib import Path

# Add the parent directory to Python path to access the src module
sys.path.append(str(Path.cwd().parent))

from src.rssam.classify import *

In [ ]:
data_dir = Path.cwd().parent / "data"

models_dir = Path.cwd().parent / "clf_models"

In [ ]:
# Make s1 VH image smaller for repo storage
input_path = data_dir / "s1_vh_only_may2025.tif"
output_path = data_dir / "output_VH_uint16.tif"
vmin, vmax = -30.0, 0.0

with rio.open(input_path) as src:
    # copy profile & set up for uint16 output
    profile = src.profile.copy()
    profile.update(dtype=rio.uint16, nodata=0, compress="lzw", count=src.count)

    vh = src.read(masked=True).astype(np.float32)
    mask = vh.mask

    input_band_names = src.descriptions

# clip to [vmin, vmax]
vh_clipped = np.clip(vh.data, vmin, vmax)

# scale so 1-1000
scale_factor = (1000 - 1) / (vmax - vmin)
vh_scaled = ((vh_clipped - vmin) * scale_factor + 1).round().astype(np.uint16)

# restore no‑data pixels to 0
vh_scaled[mask] = 0

# write out uint16 GeoTIFF and keep band names
with rio.open(output_path, "w", **profile) as dst:
    dst.write(vh_scaled)
    # set each band desc to match input
    for idx, name in enumerate(input_band_names, start=1):
        if name is not None:
            dst.set_band_description(idx, name)

In [ ]:
# Stack the S1 VH bands onto the S2 analysis image

s2_path = (
    data_dir
    / "S2C_20250516_latn563lonw0021_T30VWH_ORB080_20250516122950_compressed_downscaled.tif"
)
s1_path = data_dir / "output_VH_uint16.tif"
output_path = data_dir / "s2_plus_s1_stack.tif"

# Get S2 metadata
with rio.open(s2_path) as src_s2:
    s2_meta = src_s2.profile.copy()
    s2_transform = src_s2.transform
    s2_crs = src_s2.crs
    s2_width = src_s2.width
    s2_height = src_s2.height
    s2_descriptions = src_s2.descriptions
    s2_data = src_s2.read()

# Prepare an empty array for S1 reprojected into S2 grid
with rio.open(s1_path) as src_s1:
    s1_count = src_s1.count
    s1_dtype = src_s1.dtypes[0]
    s1_descriptions = src_s1.descriptions

# destination S1 array
s1_aligned = np.zeros((s1_count, s2_height, s2_width), dtype=s1_dtype)

# Reproject S1 to S2 grid with nearest‐neighbour
with rio.open(s1_path) as src_s1:
    reproject(
        source=rio.band(src_s1, list(range(1, s1_count + 1))),
        destination=s1_aligned,
        src_transform=src_s1.transform,
        src_crs=src_s1.crs,
        dst_transform=s2_transform,
        dst_crs=s2_crs,
        resampling=Resampling.nearest,
    )

# Stack S2 and now aligned S1
stacked = np.vstack([s2_data, s1_aligned])

# Update metadata and write out
new_meta = s2_meta.copy()
new_meta.update({"count": stacked.shape[0], "dtype": s1_dtype, "compress": "lzw"})

with rio.open(output_path, "w", **new_meta) as dst:
    dst.write(stacked)
    # keep band desc
    all_descriptions = list(s2_descriptions) + list(s1_descriptions)
    for idx, desc in enumerate(all_descriptions, start=1):
        if desc:
            dst.set_band_description(idx, desc)

print(f"Written stack to {output_path} (shape {stacked.shape})")

### Random forest classification
#### First test accuracy of predicting 5 classes using K-folds

In [ ]:
import warnings

warnings.filterwarnings(
    "ignore", message="Warning: 'partition' will ignore the 'mask' of the MaskedArray."
)


SOURCE_IMAGE = (
    data_dir
    / "S2C_20250516_latn563lonw0021_T30VWH_ORB080_20250516122950_compressed_downscaled.tif"
)

# Read the classified GeoJSON
print("Reading master_classified.geojson...")
gdf = gpd.read_file(data_dir / "master_classified.geojson")
print(f"Original polygons: {len(gdf)}")

print(gdf.crs)


# Drop any invalid geometries and empty polygons
print("Filtering valid polygons...")
gdf = gdf[gdf.geometry.is_valid]
gdf = gdf[~gdf.geometry.is_empty]
gdf = gdf[gdf.geometry.geom_type.isin(["Polygon", "MultiPolygon"])]


print(f"Class distribution:")
print(gdf["ml_class"].value_counts().sort_index())

# Load the image and extract pixels
with rio.open(SOURCE_IMAGE) as src:
    img_profile = src.profile
    img_transform = src.transform
    img_crs = src.crs

    # Extract pixels for each polygon
    X_list = []
    y_list = []

    for idx, row in gdf.iterrows():
        try:
            # Mask the image with the polygon
            geom = [row.geometry]
            masked_img, masked_transform = mask(src, geom, crop=True, filled=False)

            # Get valid pixels (not masked)
            valid_mask = ~masked_img.mask[0]  # First band mask

            if np.any(valid_mask):
                # Extract pixel values for all bands
                pixels = masked_img[:, valid_mask].T

                # We're working with the median per polygon
                pixels_med = np.median(pixels, axis=0)

                # Give pixels labels in gdf ml_class column
                label = int(row["ml_class"])

                X_list.append(pixels_med.reshape(1, -1))
                y_list.append(label)

        except Exception as e:
            print(f"Error processing polygon {idx}: {e}")
            continue

    if len(X_list) == 0:
        print("No valid pixels extracted!")
        exit()

    # Combine all pixels
    X = np.vstack(X_list)
    X = calculate_indices(X)
    y = np.hstack(y_list)

print(f"\nTotal pixels extracted: {len(X)}")
print(f"Features (bands): {X.shape[1]}")
print(f"Class distribution in extracted pixels:")
unique, counts = np.unique(y, return_counts=True)
for class_id, count in zip(unique, counts):
    print(f"  Class {class_id}: {count} pixels ({count/len(y)*100:.1f}%)")
    # Implement 5-fold cross-validation

    print("\nImplementing 5-fold cross-validation...")
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_accuracies = []
    cv_predictions = []
    cv_true_labels = []
    feature_importances = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        print(f"\nFold {fold + 1}/5")

        # Split data for this fold
        X_train_fold = X[train_idx]
        X_val_fold = X[val_idx]
        y_train_fold = y[train_idx]
        y_val_fold = y[val_idx]

        # Train model for this fold
        rf_fold = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)

        rf_fold.fit(X_train_fold, y_train_fold)

        # Make predictions
        y_pred_fold = rf_fold.predict(X_val_fold)

        # Calculate accuracy for this fold
        fold_accuracy = accuracy_score(y_val_fold, y_pred_fold)
        cv_accuracies.append(fold_accuracy)

        # Store predictions and true labels for overall evaluation
        cv_predictions.extend(y_pred_fold)
        cv_true_labels.extend(y_val_fold)

        # Store feature importances
        feature_importances.append(rf_fold.feature_importances_)

        print(f"Fold {fold + 1} accuracy: {fold_accuracy:.4f}")

    # Calculate average accuracy and standard deviation
    mean_accuracy = np.mean(cv_accuracies)
    std_accuracy = np.std(cv_accuracies)

    print(f"\n5-Fold Cross-Validation Results:")
    print(f"Mean accuracy: {mean_accuracy:.4f} (+/- {std_accuracy * 2:.4f})")
    print(f"Individual fold accuracies: {[f'{acc:.4f}' for acc in cv_accuracies]}")

    # Calculate average feature importance across folds
    mean_feature_importance = np.mean(feature_importances, axis=0)
    feature_names = [f"Band_{i+1}" for i in range(X.shape[1])]
    importance_df_cv = pd.DataFrame(
        {"Feature": feature_names, "Importance": mean_feature_importance}
    ).sort_values("Importance", ascending=False)

    print(f"\nAverage Feature Importance across 5 folds:")
    print(importance_df_cv)

    # Overall classification report using all CV predictions
    print(f"\nOverall Classification Report (5-fold CV):")
    print(classification_report(cv_true_labels, cv_predictions))

    # Overall confusion matrix
    print(f"\nOverall Confusion Matrix (5-fold CV):")
    cm_cv = confusion_matrix(cv_true_labels, cv_predictions)
    print(cm_cv)

#### Train the model on all data and write as pickle to clf_models dir

In [ ]:
rf = RandomForestClassifier(
    n_estimators=50,
    random_state=42,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    n_jobs=-1,
)

rf.fit(X, y)

model_path = models_dir / "random_forest_classifer_20250717.pkl"
with open(model_path, "wb") as f:
    pickle.dump(rf, f)

print(f"Model saved to: {model_path}")

#### Exploration - box plots per image band for the 5 classes

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create a DataFrame with X data and y labels for easier plotting
df_plot = pd.DataFrame(X)
df_plot["class"] = y

# Get the number of columns in X
n_features = X.shape[1]

# Create feature names (you can customize these based on your actual feature names)
feature_names = [
    "Blue",
    "Green",
    "Red",
    "RE_B6",
    "NIR_B8",
    "SWIR1",
    "SWIR2",
    "NDVI",
    "NDWI",
    "NBR",
    "EVI2",
    "NDVI_RE",
]

# If more features than names, pad with generic names
if n_features > len(feature_names):
    feature_names.extend(
        [f"Feature_{i}" for i in range(len(feature_names), n_features)]
    )

# Create subplots - adjust the layout based on number of features
n_cols = 3
n_rows = (n_features + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4 * n_rows))
fig.suptitle("Box Plots of Features by Class", fontsize=16)

# Flatten axes array for easier indexing
if n_rows == 1:
    axes = axes.reshape(1, -1)
axes = axes.flatten()

# Create box plots for each feature
for i in range(n_features):
    ax = axes[i]

    # Create box plot grouped by class
    data_by_class = [df_plot[df_plot["class"] == cls][i] for cls in np.unique(y)]

    bp = ax.boxplot(data_by_class, labels=np.unique(y), patch_artist=True)

    # Color the boxes
    colors = plt.cm.Set3(np.linspace(0, 1, len(np.unique(y))))
    for patch, color in zip(bp["boxes"], colors):
        patch.set_facecolor(color)

    ax.set_title(feature_names[i])
    ax.set_xlabel("Class")
    ax.set_ylabel("Value")
    ax.grid(True, alpha=0.3)

# Hide empty subplots
for i in range(n_features, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.savefig("feature_boxplots_by_class.png", dpi=300, bbox_inches="tight")
print("Plot saved to: feature_boxplots_by_class.png")

In [ ]:
# Reference class description
class_desc = {
    0: "Water",
    1: "Crop / grass",
    2: "New crop",
    3: "Bare ground",
    4: "Mature crop",
    5: "Forest",
}